In [2]:
import pandas as pd
import numpy as np
from pandas_datareader import data
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.express as px
import math
import six
import sys
sys.modules['sklearn.externals.six'] = six

import mlrose

import yfinance as yf
yf.pdr_override()

dataset = pd.read_csv('db/acoes1.csv')
dataset


,Date,VALE3.SA,RECV3.SA,KEPL3.SA,CURY3.SA,VULC3.SA,LEVE3.SA,GGBR4.SA,LREN3.SA,ALSO3.SA,...,CMIG4.SA,ITSA4.SA,BBAS3.SA,TASA4.SA,SLCE3.SA,B3SA3.SA,PETR4.SA,SUZB3.SA,KLBN4.SA,BOVA11.SA
0,2021-05-05,110.709999,15.610000,8.475000,9.85,7.810000,24.430000,32.809525,37.627274,26.660000,...,9.307692,8.839826,29.260000,25.250000,43.818180,17.166666,23.830000,67.669998,5.52,115.150002
1,2021-05-06,115.050003,16.000000,8.896666,9.92,8.020000,24.280001,34.304760,38.018181,27.110001,...,9.223076,8.831168,29.209999,26.209999,43.438015,16.933332,23.500000,67.459999,5.52,115.430000
2,2021-05-07,115.449997,16.600000,8.858333,10.09,8.380000,24.469999,34.638096,39.599998,28.020000,...,9.238461,8.952380,29.940001,25.850000,43.214874,17.780001,24.379999,67.419998,5.51,117.459999
3,2021-05-10,114.690002,15.880000,8.895000,10.25,8.170000,23.870001,34.647617,39.900002,28.309999,...,9.115384,9.047619,30.330000,24.500000,44.363636,17.766666,24.700001,68.470001,5.52,117.139999
4,2021-05-11,118.720001,16.150000,8.913333,10.20,8.370000,24.469999,35.857143,40.463634,28.510000,...,9.238461,9.047619,30.549999,23.969999,46.685951,17.616667,25.150000,68.320000,5.57,118.129997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,2023-10-16,67.300003,22.190001,10.760000,15.10,18.860001,40.130001,22.540001,12.960000,23.530001,...,12.230000,9.030000,49.810001,15.530000,37.740002,11.740000,36.680000,57.369999,4.70,112.940002
613,2023-10-17,67.849998,21.660000,10.600000,14.65,18.590000,39.970001,22.330000,12.810000,23.230000,...,12.010000,8.840000,49.360001,15.790000,37.060001,11.270000,37.669998,57.470001,4.75,112.339996
614,2023-10-18,65.360001,21.790001,10.440000,14.29,18.540001,38.900002,21.660000,12.240000,21.990000,...,11.680000,8.750000,49.119999,15.480000,36.310001,11.130000,38.520000,56.080002,4.66,110.370003
615,2023-10-19,64.419998,21.799999,10.420000,14.47,18.580000,38.759998,21.629999,12.340000,21.959999,...,11.780000,8.800000,49.500000,15.470000,36.689999,11.000000,38.340000,55.980000,4.65,110.599998


In [3]:
dinheiro_total = 5000
taxa_selic_2020 = 12
taxa_selic_2021 = 13.75
taxa_selic_2022 = 13.75
taxa_selic_2023 = 13.25
valor_2020 = dinheiro_total + (dinheiro_total * taxa_selic_2020 / 100)
valor_2021 = valor_2020 + (dinheiro_total * taxa_selic_2021 / 100)
valor_2022 = valor_2021 + (dinheiro_total * taxa_selic_2022 / 100)
valor_2023 = valor_2022 + (dinheiro_total * taxa_selic_2023 / 100)
valor_2023
taxa_selic_historico = np.array([taxa_selic_2020, taxa_selic_2021, taxa_selic_2022, taxa_selic_2023])
dataset_original = dataset.copy()
dinheiro_total = 5000
sem_risco = taxa_selic_historico.mean() / 100

# (dataset['taxa de retorno'].mean() - taxa_selic_historico.mean()/100)/ dataset['taxa de retorno'].std() * math.sqrt(246) #sharp ratio validando historico da selic


In [4]:
def fitness_function(solucao):
  dataset = dataset_original.copy()
  pesos = solucao / solucao.sum()

  for i in dataset.columns[1:]:
    dataset[i] = (dataset[i] / dataset[i][0])

  for i, acao in enumerate(dataset.columns[1:]):
    dataset[acao] = dataset[acao] * pesos[i] * dinheiro_total

  dataset.drop(labels = ['Date'], axis = 1, inplace=True)
  dataset['soma valor'] = dataset.sum(axis = 1)
  dataset['taxa retorno'] = 0.0

  for i in range(1, len(dataset)):
    dataset['taxa retorno'][i] = ((dataset['soma valor'][i] / dataset['soma valor'][i - 1]) - 1) * 100

  sharpe_ratio = (dataset['taxa retorno'].mean() - sem_risco) / dataset['taxa retorno'].std() * np.sqrt(246)

  return sharpe_ratio

In [6]:
def alocacao_ativos(dataset, dinheiro_total, seed = 0, melhores_pesos = []):
    if seed != 0:
        np.random.seed(seed)

    dataset = dataset.copy()

    if len(melhores_pesos) > 0:
        pesos = melhores_pesos
    else:  
        pesos = np.random.random(len(dataset.columns) - 1)
    #print(pesos, pesos.sum())
        pesos = pesos / pesos.sum()
    #print(pesos, pesos.sum())
    
 
    colunas = dataset.columns[1:]

    for i in colunas:
        dataset[i] = dataset[i] / dataset[i][0]

    for i, acao in enumerate(dataset.columns[1:]):
        dataset[acao] = dataset[acao] * pesos[i] * dinheiro_total

    dataset['SOMA VALOR'] = dataset[colunas].sum(axis=1)

    datas = dataset['Date']

    dataset.drop(columns=['Date'], inplace=True)
    dataset['taxa de retorno'] = 0.0

    for i in range(1, len(dataset)):
        dataset['taxa de retorno'][i] = ((dataset['SOMA VALOR'][i] / dataset['SOMA VALOR'][i-1]) - 1 )*100

    acoes_pesos = pd.DataFrame(data= {'Ações': colunas, 'Pesos': pesos*100})

    return dataset, datas, acoes_pesos , dataset.loc[len(dataset)-1]['SOMA VALOR']

In [7]:
np.random.seed(10)
pesos = np.random.random(len(dataset_original.columns) - 1)
pesos = pesos / pesos.sum()
pesos

array([0.06163703, 0.00165831, 0.05063548, 0.05983769, 0.03983621,
       0.01796373, 0.01582741, 0.06077479, 0.01351382, 0.00705933,
       0.05476781, 0.07618665, 0.00031551, 0.04092981, 0.06493738,
       0.04894759, 0.05767621, 0.02332412, 0.07334028, 0.05710249,
       0.04335528, 0.01136096, 0.02983405, 0.05387071, 0.03530735])

In [15]:
def visualiza_alocacao(solucao):
  colunas = dataset_original.columns[1:]
  for i in range(len(solucao)):
    print(colunas[i], solucao[i] * 100)
visualiza_alocacao(pesos)    

VALE3.SA 6.163702935377945
RECV3.SA 0.1658309709430696
KEPL3.SA 5.063548499195886
CURY3.SA 5.983769174487785
VULC3.SA 3.983621029540306
LEVE3.SA 1.7963732152342065
GGBR4.SA 1.582740811574609
LREN3.SA 6.077479481648798
ALSO3.SA 1.3513821635968375
DIRR3.SA 0.7059325802936983
MILS3.SA 5.476781104123487
RANI3.SA 7.618664712023379
CAMB3.SA 0.03155100407004678
CSUD3.SA 4.092981284590095
CLSC4.SA 6.493738045790383
CMIG4.SA 4.894759072079119
ITSA4.SA 5.767621296686251
BBAS3.SA 2.3324118106453207
TASA4.SA 7.334027816225093
SLCE3.SA 5.710248898596058
B3SA3.SA 4.335528087930738
PETR4.SA 1.1360955361093648
SUZB3.SA 2.983404578515426
KLBN4.SA 5.387071353909555
BOVA11.SA 3.5307345368125547


In [9]:
fitness = mlrose.CustomFitness(fitness_function)

In [18]:
problema_maximizacao = mlrose.ContinuousOpt(length=25, fitness_fn=fitness,
                                            maximize = True, min_val = 0.1, max_val = 1)

In [11]:
problema_minimizacao = mlrose.ContinuousOpt(length=25, fitness_fn=fitness,
                                            maximize = False, min_val = 0, max_val = 1)

In [19]:
melhor_solucao, melhor_custo = mlrose.hill_climb(problema_maximizacao, random_state = 1) ## hill climb
melhor_solucao, melhor_custo

(array([0.1, 1. , 0.1, 1. , 1. , 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
        0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 1. , 0.1, 0.1, 0.1]),
 -0.5607391144131273)

In [20]:
melhor_solucao = melhor_solucao / melhor_solucao.sum()
melhor_solucao, melhor_solucao.sum()

(array([0.01639344, 0.16393443, 0.01639344, 0.16393443, 0.16393443,
        0.01639344, 0.01639344, 0.01639344, 0.01639344, 0.01639344,
        0.01639344, 0.01639344, 0.01639344, 0.01639344, 0.01639344,
        0.01639344, 0.01639344, 0.01639344, 0.01639344, 0.01639344,
        0.01639344, 0.16393443, 0.01639344, 0.01639344, 0.01639344]),
 1.0)

In [21]:
visualiza_alocacao(melhor_solucao)

VALE3.SA 1.639344262295082
RECV3.SA 16.39344262295082
KEPL3.SA 1.639344262295082
CURY3.SA 16.39344262295082
VULC3.SA 16.39344262295082
LEVE3.SA 1.639344262295082
GGBR4.SA 1.639344262295082
LREN3.SA 1.639344262295082
ALSO3.SA 1.639344262295082
DIRR3.SA 1.639344262295082
MILS3.SA 1.639344262295082
RANI3.SA 1.639344262295082
CAMB3.SA 1.639344262295082
CSUD3.SA 1.639344262295082
CLSC4.SA 1.639344262295082
CMIG4.SA 1.639344262295082
ITSA4.SA 1.639344262295082
BBAS3.SA 1.639344262295082
TASA4.SA 1.639344262295082
SLCE3.SA 1.639344262295082
B3SA3.SA 1.639344262295082
PETR4.SA 16.39344262295082
SUZB3.SA 1.639344262295082
KLBN4.SA 1.639344262295082
BOVA11.SA 1.639344262295082


In [22]:
_, _, _, soma_valor = alocacao_ativos(pd.read_csv('db/acoes1.csv'), 5000, melhores_pesos=melhor_solucao)
soma_valor

7259.547587178271

In [56]:
## Simulated annealing
melhor_solucao, melhor_custo = mlrose.simulated_annealing(problema_maximizacao, random_state = 1) 
melhor_solucao = melhor_solucao / melhor_solucao.sum()
melhor_solucao, melhor_custo

(array([0.        , 0.08983183, 0.        , 0.08983183, 0.12833118,
        0.10168175, 0.        , 0.        , 0.01283312, 0.03849935,
        0.10266494, 0.02566624, 0.12833118, 0.        , 0.        ,
        0.08983183, 0.        , 0.07699871, 0.        , 0.        ,
        0.        , 0.10266494, 0.01283312, 0.        , 0.        ]),
 -0.4644106218719702)

In [57]:
visualiza_alocacao(melhor_solucao)

VALE3.SA 0.0
RECV3.SA 8.983182538678806
KEPL3.SA 0.0
CURY3.SA 8.983182538678806
VULC3.SA 12.833117912398292
LEVE3.SA 10.16817461321195
GGBR4.SA 0.0
LREN3.SA 0.0
ALSO3.SA 1.2833117912398295
DIRR3.SA 3.849935373719488
MILS3.SA 10.266494329918634
RANI3.SA 2.56662358247966
CAMB3.SA 12.833117912398292
CSUD3.SA 0.0
CLSC4.SA 0.0
CMIG4.SA 8.983182538678806
ITSA4.SA 0.0
BBAS3.SA 7.699870747438976
TASA4.SA 0.0
SLCE3.SA 0.0
B3SA3.SA 0.0
PETR4.SA 10.266494329918634
SUZB3.SA 1.2833117912398309
KLBN4.SA 0.0
BOVA11.SA 0.0


In [58]:
_, _, _, soma_valor = alocacao_ativos(pd.read_csv('db/acoes1.csv'), 5000, melhores_pesos=melhor_solucao)
soma_valor

7845.953236518406

In [12]:
#algoritmo genético 
problema_maximizacao_ag = mlrose.ContinuousOpt(length = 25, fitness_fn = fitness, 
                                               maximize = True, min_val = 0, max_val = 1)
melhor_solucao, melhor_custo = mlrose.genetic_alg(problema_maximizacao_ag, random_state = 1, pop_size=1200, mutation_prob= 0.1, max_attempts = 20)
melhor_solucao = melhor_solucao / melhor_solucao.sum()
melhor_solucao, melhor_custo
# (array([0.03344146, 0.10065911, 0.05203524, 0.05009186, 0.08986174,
#         0.05894185, 0.00878224, 0.02098791, 0.00120743, 0.01101616,
#         0.03366619, 0.01289593, 0.06706893, 0.09493909, 0.00703658,
#         0.02083494, 0.02857037, 0.0379588 , 0.01738156, 0.03688446,
#         0.0131437 , 0.10263992, 0.07054015, 0.01705707, 0.01235731]),
#  -0.9230971670710331)

(array([0.02879424, 0.05579771, 0.0324258 , 0.03998232, 0.07813185,
        0.06277506, 0.00758672, 0.00149477, 0.07428612, 0.06539574,
        0.06469392, 0.00246649, 0.07454102, 0.01268971, 0.00835557,
        0.02242307, 0.010764  , 0.08001379, 0.02257187, 0.01247531,
        0.037024  , 0.07519654, 0.04610874, 0.03970603, 0.04429961]),
 -0.869826683835935)

In [16]:
visualiza_alocacao(melhor_solucao)

VALE3.SA 2.879424224586429
RECV3.SA 5.57977128479196
KEPL3.SA 3.242579592817606
CURY3.SA 3.998231881699896
VULC3.SA 7.813184796027955
LEVE3.SA 6.2775059337832335
GGBR4.SA 0.758672315426725
LREN3.SA 0.14947696572860816
ALSO3.SA 7.428611786184745
DIRR3.SA 6.539574231400465
MILS3.SA 6.469391691825817
RANI3.SA 0.24664887886964235
CAMB3.SA 7.454101587569416
CSUD3.SA 1.2689711590959787
CLSC4.SA 0.8355572685753349
CMIG4.SA 2.2423069768949992
ITSA4.SA 1.0764000951455517
BBAS3.SA 8.001378573641665
TASA4.SA 2.257187086038771
SLCE3.SA 1.2475309300075998
B3SA3.SA 3.7024001998616054
PETR4.SA 7.519654408619086
SUZB3.SA 4.6108740989574155
KLBN4.SA 3.9706030542993487
BOVA11.SA 4.429960978150149


In [17]:
_, _, _, soma_valor = alocacao_ativos(pd.read_csv('db/acoes1.csv'), 5000, melhores_pesos=melhor_solucao)
soma_valor

6506.230863731235